In [141]:
import numpy as np
import torch
import torch.nn as nn
import torchvision
from torchvision import datasets, transforms
import matplotlib.pyplot as plt
import time
import os
import copy
import math

from utilities import *

print("PyTorch Version:", torch.__version__)
print("Torchvision Version:", torchvision.__version__)
print("GPU is available?", torch.cuda.is_available())

PyTorch Version: 1.11.0+cu113
Torchvision Version: 0.12.0+cu113
GPU is available? True


In [142]:
dtype = torch.float
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Imported datasets
For the testing and comparison of our algorithms we will use the following datasets:

1. MNIST
2. FashionMNIST
3. CIFAR10

In [143]:
ts = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0,), (1,))])

# change the flag to choose the dataset to work with
dataset_flag = 1

if dataset_flag ==0:
  trainset = datasets.MNIST('../data', train=True, download=True, transform=ts)
  testset = datasets.MNIST(root='../data', train=False, download=True, transform=ts)
elif dataset_flag ==1:
  trainset = datasets.FashionMNIST('../data', train=True, download=True, transform=ts)
  testset = datasets.FashionMNIST(root='../data', train=False, download=True, transform=ts)
else:
  trainset = datasets.CIFAR10('../data', train=True, download=True, transform=ts)
  testset = datasets.CIFAR10(root='../data', train=False, download=True, transform=ts)

In [144]:
"""
from sklearn import datasets
from sklearn.model_selection import train_test_split

boston = datasets.load_boston()
X = boston.data
y = boston.target
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

x_train = torch.from_numpy(x_train).to(device = device)
x_test = torch.from_numpy(x_test).to(device = device)
y_train = torch.from_numpy(y_train).to(device = device)
y_test = torch.from_numpy(y_test).to(device = device)

x_train = x_train.T.float()
x_test = x_test.T.float()

N = x_train.shape[1]
N_test = x_test.shape[1]
"""

'\nfrom sklearn import datasets\nfrom sklearn.model_selection import train_test_split\n\nboston = datasets.load_boston()\nX = boston.data\ny = boston.target\nx_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)\n\nx_train = torch.from_numpy(x_train).to(device = device)\nx_test = torch.from_numpy(x_test).to(device = device)\ny_train = torch.from_numpy(y_train).to(device = device)\ny_test = torch.from_numpy(y_test).to(device = device)\n\nx_train = x_train.T.float()\nx_test = x_test.T.float()\n\nN = x_train.shape[1]\nN_test = x_test.shape[1]\n'

# Dataset preprocessing

In [145]:
x_train, y_train, x_test, y_test, y_train_one_hot, y_test_one_hot= load_dataset(trainset, testset, 10)

# We move to GPU
x_train = x_train.to(device = device)
x_test = x_test.to(device = device)
y_train = y_train.to(device = device)
y_test = y_test.to(device = device)
y_train_one_hot = y_train_one_hot.to(device = device)
y_test_one_hot = y_test_one_hot.to(device = device)

# Architecture initialization

For the MultiLayerPerceptron we have the parameters **input_size** , **hidden_size**,**output_size** corresponding to the size of the input layer, the hidden layer and the output layer, respectively.

The MLP only has 3 layers like https://github.com/timlautk/BCD-for-DNNs-PyTorch/blob/master/bcd_dnn_mlp_mnist.ipynb as a starting point.

Also we use ReLU currently for the same reason.

In [146]:
input_size = x_train.shape[0]
hidden_size = 1500
output_size = 10

# Training

Note: Fix it so that it moves everything to device in the following function and that it does the label sample split here

In [147]:
def update_v_js(U1, U2, W, b, rho, gamma):
    """
    The function updates the V_js parameters during the training phase
    
    :param U1: The U parameter on the same level of V that we are updating
    :param U2: The U parameter which is in the next level of the V that we are updating
    :param W: The W parameter which is in the next level of the V that we are updating
    :param b: The b parameter which is in the next level of the V that we are updating
    :param rho: The constant rho parameter which is in the next level of the V that we are updating
    :param gamma: The constant gamma parameter which is in the next level of the V that we are updating
    :return: The updated V
    """
    _, d = W.size()
    I = torch.eye(d, device=device)
    U1 = nn.ReLU()(U1)
    _, col_U2 = U2.size()
    Vstar = torch.mm(torch.inverse(rho * (torch.mm(torch.t(W), W)) + gamma * I),
                     rho * torch.mm(torch.t(W), U2 - b.repeat(1, col_U2)) + gamma * U1)
    return Vstar

In [148]:
def update_wb_js(U, V, W, b, alpha, rho):
    """
    The function updates the W and b parameters during the training phase
    
    :param U: The U in the current level of W and b
    :param V: The V in the previous level with respect to the W that we are updating
    :param W: The current W that we have to update
    :param b: The current b that we have to update
    :param alpha: The alpha constant of the updates
    :param rho: The rho constant of the updates
    :return:
    """
    d, N = V.size()
    I = torch.eye(d, device=device)
    _, col_U = U.size()
    Wstar = torch.mm(alpha * W + rho * torch.mm(U - b.repeat(1, col_U), torch.t(V)),
                     torch.inverse(alpha * I + rho * (torch.mm(V, torch.t(V)))))
    bstar = (alpha * b + rho * torch.sum(U - torch.mm(W, V), dim=1).reshape(b.size())) / (rho * N + alpha)
    return Wstar, bstar

In [149]:
def relu_prox(a, b, gamma, d, N):
    """
    The function compute the solution to the relu proximal update problem
    
    :param a: the a in the closed formula of the linearized update
    :param b: the b in the closed formula of the linearized update
    :param gamma: The constant used in the update
    :param d: the dimension of the current layer
    :param N: The number of samples
    :return: The obtained solution of the prox update
    """
    val = torch.empty(d, N, device=device)
    x = (a + gamma * b) / (1 + gamma)
    y = torch.min(b, torch.zeros(d, N, device=device))
    val = torch.where(a + gamma * b < 0, y, torch.zeros(d, N, device=device))
    val = torch.where(
        ((a + gamma * b >= 0) & (b >= 0)) | ((a * (gamma - np.sqrt(gamma * (gamma + 1))) <= gamma * b) & (b < 0)), x,
        val)
    val = torch.where((-a <= gamma * b) & (gamma * b <= a * (gamma - np.sqrt(gamma * (gamma + 1)))), b, val)
    return val


In [156]:
def make_pred(Ws,bs,input,N):
  """
  The function is used to make the predictions based on the best found parameters
  :param Ws: The weight matrices
  :param bs: the bias vectors
  :return pred, prob
  """
  a1_train = input
  for i in range(0,len(Ws)-1):
    a1_train = nn.ReLU()(torch.addmm(bs[i].repeat(1, N), Ws[i], a1_train))
  pred = torch.argmax(torch.addmm(bs[len(Ws)-1].repeat(1, N), Ws[len(Ws)-1], a1_train), dim=0)
  output_last = torch.addmm(bs[len(Ws)-1].repeat(1, N), Ws[len(Ws)-1], a1_train)
  prob = torch.exp(output_last)/torch.sum(torch.exp(output_last),dim=0)
  return pred, prob

In [157]:
from torch.nn.functional import cross_entropy
cross_entropy = nn.CrossEntropyLoss()

In [165]:
#The function requires at least 1 hidden layer otherwise it need some rewrriting
def execute_training(layers, input_size, hidden_size, output_size, train_set, val_set, 
                     train_labels, val_labels, use_gradient,
                     niter = 100, gamma = 1, alpha = 5):
  """
  The function takes the following arguements and produces a list of weights and biases with which 
  you can use the make_pred function to get a list of predictions
  :param layers: The total number of layers of the network
  :param input_size: The total size of the input layer
  :param hidden_size: The size of the hidden layer
  :param output_size: The size of the output layer (usefull for multiclass classification)
  :param train_set: The training set
  :param val_set: The validation set
  :param train_labels: The training labels
  :param val labels: The validation labels
  :param use_gradient: True if the first update of V is carried out without linearization but using the gradient
  :param niter: The default number of epochs to train the network
  :param gamma: The gamma parameter of the algorithm
  :param alpha: The alpha parameter of the algorithm
  :return Ws,bs: Returns two lists that go in order from the input to the output layer of the weights and the biases of each layer
  """

  N = len(train_labels)
  N_test = len(val_labels)

  # weight initialization (we replicate pytorch weight initialization)

  std = math.sqrt(1/input_size)
  W = torch.FloatTensor(hidden_size, input_size).uniform_(-std, std)
  b = torch.FloatTensor(hidden_size, 1).uniform_(-std, std)

  b = b.to(device = device)
  W = W.to(device = device)

  U = torch.addmm(b.repeat(1, N), W, x_train) # equivalent to W1@x_train+b1.repeat(1,N)
  V = nn.ReLU()(U)

  Ws = [W]
  bs = [b]
  Us = [U]
  Vs = [V]

  for i in range(1,layers-1):
    std = math.sqrt(1/hidden_size)
    W = torch.FloatTensor(hidden_size, hidden_size).uniform_(-std, std)
    b = torch.FloatTensor(hidden_size, 1).uniform_(-std, std)
    b = b.to(device = device)
    W = W.to(device = device)
    U = torch.addmm(b.repeat(1, N), W, Vs[-1])
    V = nn.ReLU()(U)
    Ws.append(W)
    bs.append(b)
    Us.append(U)
    Vs.append(V)
  
  std = math.sqrt(1/hidden_size)
  W = torch.FloatTensor(output_size, hidden_size).uniform_(-std, std)
  b = torch.FloatTensor(output_size, 1).uniform_(-std, std)

  # we move them to GPU
  b = b.to(device = device)
  W = W.to(device = device)
  U = torch.addmm(b.repeat(1, N), W, Vs[-1])
  V = U
  Ws.append(W)
  bs.append(b)
  Us.append(U)
  Vs.append(V)
  
  # constant initialization

  gamma1 = gamma2 = gamma3 = gamma4 = gamma

  rho = gamma
  rho1 = rho2 = rho3 = rho4 = rho

  alpha1 = alpha2 = alpha3 = alpha4 = alpha5 = alpha6 = alpha7 \
  = alpha8 = alpha9 = alpha10 = alpha

  # vector of performance initialization

  loss1 = np.empty(niter)
  loss2 = np.empty(niter)
  loss_class = np.empty(niter)
  accuracy_train = np.empty(niter)
  accuracy_test = np.empty(niter)
  time1 = np.empty(niter)

  print('Train on', N, 'samples, validate on', N_test, 'samples')
  for k in range(niter):

    start = time.time()

    # update V3
    if use_gradient == True:
      if (k == 1):
        Vs[-1] = (y_train_one_hot + gamma3*Us[-1] + alpha1*Vs[-1])/(1+ gamma3 + alpha1)
      else:
        for i in range(250):
          Vs[-1] = Vs[-1] - (torch.exp(Vs[-1])/torch.sum(torch.exp(Vs[-1]),dim=0)-y_train_one_hot) * 0.01/(i+1)
    else:
      Vs[-1] = (y_train_one_hot + gamma3*Us[-1] + alpha1*Vs[-1])/(1+ gamma3 + alpha1)

    # update U3 
    Us[-1] = (gamma3*Vs[-1] + rho3*(torch.mm(Ws[-1],Vs[-2]) + bs[-1].repeat(1,N)))/(gamma3 + rho3)

    # update W3 and b3
    W, b = update_wb_js(Us[-1],Vs[-2],Ws[-1],bs[-1],alpha1, rho3)
    Ws[-1] = W
    bs[-1] = b

    for i in range(len(Vs)-2,0,-1):
      Vs[i] = update_v_js(Us[i],Us[i+1],Ws[i+1],bs[i+1],rho3,gamma2)
      Us[i] = relu_prox(Vs[i],(rho2*torch.addmm(bs[i].repeat(1,N), Ws[i], Vs[i-1]) +
                               alpha2*Us[i])/(rho2 + alpha2),(rho2 + alpha2)/gamma2, hidden_size, N)
      W,b = update_wb_js(Us[i],Vs[i-1],Ws[i],bs[i],alpha3,rho2)
      Ws[i] = W
      bs[i]= b
    
    # update V1
    Vs[0] = update_v_js(Us[0],Us[1],Ws[1],bs[1],rho2,gamma1)
        
    # update U1
    Us[0] = relu_prox(Vs[0],(rho1*torch.addmm(bs[0].repeat(1,N), Ws[0], x_train) +
                             alpha7*Us[0])/(rho1 + alpha7),(rho1 + alpha7)/gamma1, hidden_size, N)
    
    # update W1 and b1
    W, b = update_wb_js(Us[0],x_train,Ws[0],bs[0],alpha8,rho1)
    Ws[0] = W
    bs[0] = b

    #a1_train = nn.ReLU()(torch.addmm(b1.repeat(1, N), W1, x_train))
    #a1_train = x_train
    #for i in range(len(Vs)-1,0,-1):
    #  a1_train = nn.ReLU()(torch.addmm(bs[i].repeat(1, N), Ws[i], a1_train))
    #pred = torch.argmax(torch.addmm(bs[0].repeat(1, N), Ws[0], a1_train), dim=0)
    pred,_ = make_pred(Ws,bs,x_train,N)

    #a1_test = x_test
    #a1_test = nn.ReLU()(torch.addmm(b1.repeat(1, N_test), W1, x_test))
    #for i in range(len(Vs)-1,0,-1):
    #  a1_test = nn.ReLU()(torch.addmm(bs[i].repeat(1, N_test), Ws[i], a1_test))
    #pred_test = torch.argmax(torch.addmm(bs[0].repeat(1, N_test), Ws[0], a1_test), dim=0)
    pred_test, prob_test = make_pred(Ws,bs,x_test,N_test) 

    
    loss_class[k] = torch.sum(- y_test_one_hot * torch.log(prob_test))

    loss1[k] = gamma/2*torch.pow(torch.dist(Vs[-1],y_train_one_hot,2),2).cpu().numpy()
    loss2[k] = loss1[k] + gamma/2 * torch.pow(torch.dist(torch.addmm(bs[0].repeat(1,N), Ws[0], x_train),Us[0],2),2).cpu().numpy()

    for i in range(1,layers):
      loss2[k] = loss2[k] + gamma/2 * torch.pow(torch.dist(torch.addmm(bs[i].repeat(1,N), Ws[i], Vs[i-1]),Us[i],2),2).cpu().numpy()

    #loss2[k] = loss1[k] + rho1/2*torch.pow(torch.dist(torch.addmm(b1.repeat(1,N), W1, x_train),U1,2),2).cpu().numpy() \
    #+rho2/2*torch.pow(torch.dist(torch.addmm(b2.repeat(1,N), W2, V1),U2,2),2).cpu().numpy() \
    #+rho3/2*torch.pow(torch.dist(torch.addmm(b3.repeat(1,N), W3, V2),U3,2),2).cpu().numpy()
        
    # compute training accuracy
    correct_train = pred == train_labels
    accuracy_train[k] = np.mean(correct_train.cpu().numpy())
        
    # compute validation accuracy
    correct_test = pred_test == val_labels
    accuracy_test[k] = np.mean(correct_test.cpu().numpy())
        
    # compute training time
    stop = time.time()
    duration = stop - start
    time1[k] = duration
        
    # print results
    print('Epoch', k + 1, '/', niter, '\n', 
          '-', 'time:', time1[k], '-', 'sq_loss:', loss1[k], '-', 'tot_loss:',
          loss2[k], '-', 'loss_class:', loss_class[k], '-', 'acc:',
          accuracy_train[k], '-', 'val_acc:', accuracy_test[k])
    
  print('The total time spent is:', np.sum(time1), 's')
  return loss2,loss_class,accuracy_test

In [169]:
loss2, loss_class , accuracy_test = execute_training(3, input_size, hidden_size, output_size, x_train, x_test, y_train, y_test,
                                         True, niter = 100, gamma = 1, alpha = 5)

Train on 60000 samples, validate on 10000 samples
Epoch 1 / 100 
 - time: 1.254204273223877 - sq_loss: 27133.44140625 - tot_loss: 27137.89398983121 - loss_class: 22896.875 - acc: 0.20288333333333333 - val_acc: 0.2019
Epoch 2 / 100 
 - time: 1.2348954677581787 - sq_loss: 20125.6875 - tot_loss: 20153.516769926995 - loss_class: 22402.408203125 - acc: 0.5325833333333333 - val_acc: 0.5221
Epoch 3 / 100 
 - time: 1.2764136791229248 - sq_loss: 17598.59375 - tot_loss: 17610.66461057961 - loss_class: 21959.951171875 - acc: 0.7324333333333334 - val_acc: 0.7146
Epoch 4 / 100 
 - time: 1.2688531875610352 - sq_loss: 15279.66796875 - tot_loss: 15287.164010394365 - loss_class: 21551.84765625 - acc: 0.8118166666666666 - val_acc: 0.7983
Epoch 5 / 100 
 - time: 1.275266170501709 - sq_loss: 13165.2744140625 - tot_loss: 13172.195207450539 - loss_class: 21162.244140625 - acc: 0.84335 - val_acc: 0.8285
Epoch 6 / 100 
 - time: 1.2673799991607666 - sq_loss: 11251.59765625 - tot_loss: 11257.422478109598 - loss

In [167]:
y_test_one_hot.shape

torch.Size([10, 10000])

# Analyis of the results

In [ ]:
## We plot the train losses

plot_train_losses(loss1.shape[0], loss1, 'Coordinate_descent')

In [ ]:
## We plot the test accuracy

plot_test_accuracy(accuracy_test.shape[0], accuracy_test, 'Coordinate_descent')
